In [1]:
#Implementations
import numpy as np
import nibabel as nib
from nilearn import image, input_data
from scipy import stats
import networkx as nx
import community
import matplotlib.pyplot as plt
import os      
import glob



graph_attributes = np.empty((0, 8))

C:\Users\User\AppData\Roaming\Python\Python37\site-packages\nilearn\__init__.py:67: FutureWarning: Python 3.7 support is deprecated and will be removed in release 0.12 of Nilearn. Consider switching to Python 3.9 or 3.10.
  _python_deprecation_warnings()
C:\Users\User\AppData\Roaming\Python\Python37\site-packages\nilearn\input_data\__init__.py:23: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


### Main method

In [ ]:
import glob
import nibabel as nib

# Define the folder path where the .nii files are located
folder_path = "nifti_files"

# Use glob to get a list of .nii files in the folder
nii_files = glob.glob(folder_path + "/*.nii.gz")

# Iterate over the .nii files and read them using nibabel
for nii_file in nii_files:
    try:
        # Load the NIfTI image
        image = nib.load(nii_file)

        # Process the image and calculate graph attributes
        partial_correlation_matrix = partial_corr(image)
        threshold_matrix = threshold(partial_correlation_matrix)
        graph_attr = graph_attributesfunc(threshold_matrix)
        communities_neki = communitiesnum(threshold_matrix)
        graph_attr.append(communities_neki)
        graph_attr.append(nii_file)
        graph_attributes = np.append(graph_attributes, [graph_attr], axis=0)
        embedding_modelar = embedding_model(threshold_matrix, nii_file)
        print(graph_attr, "added")
#[num_nodes, num_edges, avg_degree, lcc_percentage, avg_clustering]
    except Exception as e:
        print(f"Error loading file: {nii_file}")
        print(e)

### Brain parcellation and partial correlation

In [10]:
def partial_corr(image_file):
    # Load the atlas image
    atlas_file = 'BN_Atlas_246_1mm.nii.gz'  # Replace with your atlas file
    atlas_data = nib.load(atlas_file).get_fdata()

    # Create a masker to extract ROI signals
    masker = input_data.NiftiLabelsMasker(labels_img=atlas_file, standardize=True)

    # Extract ROI signals from the image data
    roi_signals = masker.fit_transform(image_file)
    # Compute the partial correlation matrix
    partial_corr_matrix = np.zeros((len(roi_signals.T), len(roi_signals.T)))
    for i in range(len(roi_signals.T)):
        for j in range(i+1, len(roi_signals.T)):
            # Calculate the partial correlation coefficient
            r_partial, _ = stats.pearsonr(roi_signals.T[i], roi_signals.T[j])

            # Perform Fisher transformation to convert to a z-score
            z_partial = 0.5 * np.log((1 + r_partial) / (1 - r_partial))

            # Calculate the p-value
            p_value = 2 * stats.norm.cdf(-np.abs(z_partial))

            # Store the partial correlation coefficient in the matrix
            partial_corr_matrix[i, j] = r_partial
            partial_corr_matrix[j, i] = r_partial
    return partial_corr_matrix

# The partial correlation matrix will have shape (n_ROIs, n_ROIs)


### Threshold

In [11]:
def threshold(partial_corr_matrix):
    # Thresholding parameters
    mean = np.mean(partial_corr_matrix)
    std = np.std(partial_corr_matrix)
    # Set the threshold value as a multiple of the standard deviation
    threshold_value2 = mean + std
    threshold_value1 = 4.0 # Set the threshold value as per your requirement
    # Thresholding the correlation matrix
    adjacency_matrix = np.where(np.abs(partial_corr_matrix) >= threshold_value2, 1, 0)  # Binary adjacency matrix
    weighted_adjacency_matrix = np.where(np.abs(partial_corr_matrix) >= threshold_value2, np.abs(partial_corr_matrix), 0)  # Weighted adjacency matrix
    return weighted_adjacency_matrix


In [12]:
def  graphmod(weighted_adjacency_matrix):
    graph = nx.from_numpy_array(weighted_adjacency_matrix)
    return graph 

### Graph attributes

In [13]:
def graph_attributesfunc(weighted_adjacency_matrix):
    # Create a graph from the weighted adjacency matrix
    graph = nx.from_numpy_array(weighted_adjacency_matrix)

    # Optional: Set the labels of the nodes if available
    node_labels = range(weighted_adjacency_matrix.shape[0])  # Assuming each row/column represents a node
    labels = {i: label for i, label in enumerate(node_labels)}
    nx.set_node_attributes(graph, labels, 'label')
    # Optional: Set the edge weights as attributes
    edge_weights = {(u, v): weight for (u, v, weight) in graph.edges(data='weight')}
    nx.set_edge_attributes(graph, edge_weights, 'weight')
    num_nodes = graph.number_of_nodes()
    num_edges = graph.number_of_edges()

    # Calculate the average degree
    avg_degree = sum(dict(graph.degree()).values()) / num_nodes

    # Calculate the largest connected component (LCC)
    lcc = max(nx.connected_components(graph), key=len)
    lcc_percentage = len(lcc) / num_nodes * 100
    # Calculate the clustering coefficient
    avg_clustering = nx.average_clustering(graph)
        # Calculate degree mixing
    degree_mixing = nx.degree_mixing_dict(graph)

    # Calculate degree assortativity
    degree_assortativity = nx.degree_assortativity_coefficient(graph, x='out', y='in')
    return [num_nodes, num_edges, avg_degree, lcc_percentage, avg_clustering, degree_assortativity]
    


### Communities

In [14]:
def communitiesnum(weighted_adjacency_matrix):
    graph = nx.from_numpy_array(weighted_adjacency_matrix)
    
    # Perform community detection using the Louvain algorithm
    partition = community.best_partition(graph)

    # Create a dictionary of nodes and their corresponding community IDs
    communities = {}
    for node, comm_id in partition.items():
        communities[node] = comm_id

    # Create a subgraph for each community
    community_subgraphs = []
    for community_id in set(communities.values()):
        nodes_in_community = [node for node, comm_id in communities.items() if comm_id == community_id]
        community_subgraph = graph.subgraph(nodes_in_community)
        community_subgraphs.append(community_subgraph)

    # Find the giant component (largest connected component)
    giant_component = max(community_subgraphs, key=len)
    community_subgraphs.append(giant_component)

    return len(community_subgraphs)



## Node2Vec

In [15]:
from nodevectors import Node2Vec
from node2vec import Node2Vec

In [ ]:
# Fit embedding model to graph
g2v = Node2Vec(
    n_components=32,
    walklen=10
)



In [ ]:
# Convert the edge weights to object data type
edge_weights = {(u, v): float(weight) for u, v, weight in graph.edges(data='weight')}
nx.set_edge_attributes(graph, edge_weights, 'weight')

In [ ]:
def embedding_model(weighted_adjacency_matrix, nii_file):
    # Create a graph from the weighted adjacency matrix
    graph = nx.from_numpy_array(weighted_adjacency_matrix)
    # Preprocess the graph for node2vec
    node2vec = Node2Vec(graph)
    # Generate walks
    walks = node2vec.walks
    # Train the node2vec model on the walks
    model = node2vec.fit()
    # Get the embeddings for all nodes
    embeddings = {str(node): model.wv[str(node)] for node in graph.nodes}
    # Query embeddings for node 42
    embedding_42 = embeddings[str(42)]
    filename = str(nii_file) + '_node2vec_embeddings.txt'
    # Save the embeddings to a file
    model.wv.save_word2vec_format(filename)


## Prediction model

In [16]:
import os
import numpy as np

folder_path = "nifti_files"  # Path to the folder containing the embedding files
embedding_dimension = 128  # Embedding dimension size

# Get a list of all files in the folder
file_list = os.listdir(folder_path)

# Filter files with the naming convention "patient_id_node2vec_embeddings"
embedding_files = [file for file in file_list if "_node2vec_embeddings" in file]

# Initialize an empty matrix to store the embeddings
embedding_matrix = np.zeros((len(embedding_files), embedding_dimension))

# Iterate over each embedding file
for i, file in enumerate(embedding_files):
    file_path = os.path.join(folder_path, file)
    
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    # Extract the embedding vectors (excluding the first line)
    embedding_vectors = []
    for line in lines[1:]:
        vector = list(map(float, line.split()[1:]))
        embedding_vectors.append(vector)
    
    # Calculate the average embedding vector
    average_embedding = np.mean(embedding_vectors, axis=0)
    
    # Store the average embedding vector in the embedding matrix
    embedding_matrix[i] = average_embedding


In [56]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

# Perform oversampling on the embedding matrix and labels
oversampler = RandomOverSampler()
labels = np.array([1,1,1,1,1,1,1, 1, 0, 0,0, 0, 1, 1, 1, 1,1,1,1])  # Example labels, replace with your actual labels
embedding_matrix_resampled, labels_resampled = oversampler.fit_resample(embedding_matrix, labels)

# Split the resampled data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(embedding_matrix_resampled, labels_resampled, test_size=0.33)

# Initialize the classifier
classifier = SVC()

# Train the model on the training data
classifier.fit(X_train, y_train)

# Predict the labels for the test set
y_pred = classifier.predict(X_test)

# Calculate precision, recall, and F1-score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)



print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)



# Create the predicted edge list
predicted_edge_list = []
for i, y in enumerate(y_pred):
    if y == 1:
        source_node = i  # Assuming that the node indices correspond to the row index in the embedding_matrix
        for j, true_y in enumerate(y_test):
            if true_y == 1 and j != i:
                target_node = j
                predicted_edge_list.append((source_node, target_node, 1.0))  # Use a constant weight of 1.0 for predicted edges

# true_network_graph = nx.from_numpy_array(threshold_matrix)
# # Compute the Mean Average Precision (MAP)
# map_val = computeMAP(predicted_edge_list, true_network_graph)  # Replace 'true_network_graph' with the actual network graph object

# print("MAP:", map_val)



Precision: 1.0
Recall: 0.8333333333333334
F1-score: 0.9090909090909091


### Logistic regression

In [54]:
#Logistic regression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# ... (previous cod
X_train, X_test, y_train, y_test = train_test_split(embedding_matrix_resampled, labels_resampled, test_size=0.25)
# Initialize the Logistic Regression classifier
logreg_classifier = LogisticRegression()

# Train the model on the training data
logreg_classifier.fit(X_train, y_train)

# Predict the labels for the test set using Logistic Regression
logreg_y_pred = logreg_classifier.predict(X_test)

# Calculate accuracy, precision, recall, and F1-score using Logistic Regression
logreg_accuracy = accuracy_score(y_test, logreg_y_pred)
logreg_precision = precision_score(y_test, logreg_y_pred)
logreg_recall = recall_score(y_test, logreg_y_pred)
logreg_f1 = f1_score(y_test, logreg_y_pred)

# Calculate ROC-AUC score using Logistic Regression
logreg_roc_auc = roc_auc_score(y_test, logreg_y_pred)

print("Logistic Regression Results:")
print("Accuracy:", logreg_accuracy)
print("Precision:", logreg_precision)
print("Recall:", logreg_recall)
print("F1-score:", logreg_f1)


Logistic Regression Results:
Accuracy: 0.75
Precision: 0.6666666666666666
Recall: 1.0
F1-score: 0.8


In [19]:
import os
import numpy as np

folder_path = "nifti_files"  # Path to the folder containing the embedding files
embedding_dimension = 128  # Embedding dimension size

# Get a list of all files in the folder
file_list = os.listdir(folder_path)

# Filter files with the naming convention "patient_id_node2vec_embeddings"
embedding_files = [file for file in file_list if "_node2vec_embeddings" in file]

# Initialize an empty matrix to store the embeddings
embedding_matrix = np.zeros((len(embedding_files), embedding_dimension))

# Iterate over each embedding file
for i, file in enumerate(embedding_files):
    file_path = os.path.join(folder_path, file)
    
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    # Extract the embedding vectors (excluding the first line)
    embedding_vectors = []
    for line in lines[1:]:
        vector = list(map(float, line.split()[1:]))
        embedding_vectors.append(vector)
    
    # Calculate the average embedding vector
    average_embedding = np.mean(embedding_vectors, axis=0)
    
    # Store the average embedding vector in the embedding matrix
    embedding_matrix[i] = average_embedding


In [58]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC  # Example classifier, replace with your desired algorithm

labels = np.array([1,1,1,1,1,1,1, 1, 1, 0,0, 0, 1, 1, 1, 1,1,1,1])  # Example labels, replace with your actual labels

X_train, X_test, y_train, y_test = train_test_split(embedding_matrix, labels, test_size=0.33)


classifier = SVC()  # Initialize the classifier
classifier.fit(X_train, y_train)  # Train the model on the training data
accuracy = classifier.score(X_test, y_test)  # Calculate the accuracy on the test set
print("Accuracy:", accuracy)



Accuracy: 0.8571428571428571


In [49]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

# Your code for loading data, splitting, and training the classifier

# Calculate predictions on the test set
y_pred = classifier.predict(X_test)

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Calculate ROC AUC score
roc_auc = roc_auc_score(y_test, y_pred)

# Print the metrics
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)



Precision: 0.7142857142857143
Recall: 1.0
F1 Score: 0.8333333333333333


In [24]:
file_list

['OHSU_0050142.nii.gz',
 'OHSU_0050142.nii.gz_node2vec_embeddings.txt',
 'OHSU_0050143.nii.gz',
 'OHSU_0050143.nii.gz_node2vec_embeddings.txt',
 'OHSU_0050144.nii.gz',
 'OHSU_0050144.nii.gz_node2vec_embeddings.txt',
 'OHSU_0050145.nii.gz',
 'OHSU_0050145.nii.gz_node2vec_embeddings.txt',
 'OHSU_0050146.nii.gz',
 'OHSU_0050146.nii.gz_node2vec_embeddings.txt',
 'OHSU_0050147.nii.gz',
 'OHSU_0050147.nii.gz_1_node2vec_embeddings.txt',
 'OHSU_0050148.nii.gz',
 'OHSU_0050148.nii.gz_1_node2vec_embeddings.txt',
 'OHSU_0050149.nii.gz',
 'OHSU_0050150.nii.gz',
 'OHSU_0050152.nii.gz',
 'Olin_0050110.nii.gz',
 'Olin_0050110.nii.gz_1_node2vec_embeddings.txt',
 'Olin_0050111.nii.gz',
 'Olin_0050111.nii.gz_1_node2vec_embeddings.txt',
 'Olin_0050112.nii.gz',
 'Olin_0050113.nii.gz',
 'Olin_0050113.nii.gz_node2vec_embeddings.txt',
 'Olin_0050114.nii.gz',
 'Olin_0050126.nii.gz',
 'Olin_0050127.nii.gz',
 'Olin_0050128.nii.gz',
 'Olin_0050128.nii.gz_1_node2vec_embeddings.txt',
 'Olin_0050129.nii.gz',
 'Olin